### Multi-node distributed inference of *Hugging Face* models with *PyTorch*, *Horovod*, and *Petastorm*

##### Here we perform model inference using the NLP model from [*Hugging Face*](https://huggingface.co/), after we fine tune it. We use [*PyTorch*](https://pytorch.org/) as the Deep Learning framework to interface with the model and [*Horovod*](https://horovod.ai/) to provide the distributed infrastructure on *Spark* to parallelize the process. [*Petastorm*](https://petastorm.readthedocs.io/en/latest/index.html) is used to integrate the distributed data layer on *Spark* with *PyTorch*.

Import the necessary packages.

In [0]:
import os
import time
import random
import pickle
import pandas as pd
import numpy as np

import torch
from torch.optim import AdamW
import transformers
from transformers import AutoConfig, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, SequentialSampler
from torch.utils.data.distributed import DistributedSampler

from petastorm.spark import SparkDatasetConverter, make_spark_converter
from petastorm import TransformSpec

import horovod.torch as hvd
from sparkdl.horovod import log_to_driver
from sparkdl import HorovodRunner

/databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py:171: FutureWarning: pyarrow.LocalFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  original_result = python_builtin_import(name, globals, locals, fromlist, level)
/databricks/python/lib/python3.9/site-packages/petastorm/spark/spark_dataset_converter.py:28: FutureWarning: pyarrow.LocalFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  from pyarrow import LocalFileSystem


Define the number of workers in the cluster and the folder in the Databricks File System to load the prepared data from.

In [0]:
num_workers = 8

test_data_folder = '/test_data'

Read the testing data into a *Spark DataFrame*.

In [0]:
sdf_tokens_test = spark.read.parquet(os.path.join(test_data_folder, 'tokens_test_data'))
display(sdf_tokens_test.select(sdf_tokens_test.columns).limit(10))

id,label,input_ids,token_type_ids,attention_mask
25769803776,1,"List(1, 379, 12902, 304, 449, 6003, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","List(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","List(1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
25769803777,1,"List(1, 311, 265, 262, 52533, 2883, 267, 1240, 323, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","List(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","List(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
25769803778,1,"List(1, 53829, 303, 7964, 299, 10300, 697, 265, 15118, 20228, 263, 26531, 3836, 265, 13868, 323, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","List(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","List(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
25769803779,1,"List(1, 262, 12711, 11351, 32938, 52099, 660, 3260, 269, 3828, 13646, 112649, 366, 263, 1917, 268, 323, 15359, 15960, 263, 342, 1376, 271, 23410, 366, 507, 79330, 1560, 66490, 366, 286, 107406, 262, 266, 8163, 1442, 4437, 2626, 366, 319, 1161, 1199, 334, 299, 1330, 264, 22959, 30318, 121152, 280, 

Here we set a folder in the Databricks File System to be used by *Petastorm* as a cache for intermediate data. We then create the [*Petastorm SparkDatasetConverter*](https://petastorm.readthedocs.io/en/latest/api.html#module-petastorm.spark.spark_dataset_converter) object that we later use for reading data from the *Spark DataFrame* into a *PyTorch DataLoader*.

In [0]:
spark.conf.set(SparkDatasetConverter.PARENT_CACHE_DIR_URL_CONF, 'file:///dbfs/tmp/petastorm/cache')

converter_test = make_spark_converter(sdf_tokens_test, dtype=None)

/databricks/python/lib/python3.9/site-packages/petastorm/fs_utils.py:88: FutureWarning: pyarrow.localfs is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  self._filesystem = pyarrow.localfs
The median size 17195 B (< 50 MB) of the parquet files is too small. Total size: 137807 B. Increase the median file size by calling df.repartition(n) or df.coalesce(n), which might help improve the performance. Parquet files: file:/dbfs/tmp/petastorm/cache/20220914170134-appid-app-20220914152732-0000-9a1ef882-4df4-4a5b-a6da-c041097e6242/part-00001-tid-8489269314896611345-7f714b63-e11e-4708-98da-cc8d001cfb3a-332-1-c000.parquet, ...


Here is where we define the main function of this notebook, which specifies the model fine tuning procedure as a *PyTorch* distributed process with *Horovod*, to be run on the worker nodes in our cluster.

In [0]:
def model_inference():

    # initiate Horovod in the worker node
    ######
    hvd.init()
    ######

    # ranks are the unique Horovod process identifiers
    # local_rank refers to the process(es) running on a node
    # rank refers to the processes running on the entire cluster (all participating nodes)
    # world_size is the size of the cluster in number of nodes in a cluster
    ######
    local_rank = hvd.local_rank()
    rank = hvd.rank()
    world_size = hvd.size()
    ######

    # rank 0 usually is the node that triggers the MPI job execution in a Horovod cluster
    # when we need to run something only in a single node in the cluster, we usually use the rank 0 node
    ######
    inference_folder = '/dbfs/inference_outputs'
    if rank == 0:
        os.makedirs(inference_folder, exist_ok = True)
        log_to_driver(f'Cluster size: {world_size} GPUs')
    ######

    # set the GPU device for PyTorch
    ######
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    if device.type == 'cuda':
        torch.cuda.set_device(hvd.local_rank())
        
    torch.cuda.empty_cache()
    ######
    
    # set random seeds for reproducibility
    ######
    seed_val = 111
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)
    ######

    # here we instantiate the pre-trained model from Hugging Face, but load it with the fine-tuned model weights
    ######
    model_type = 'microsoft/deberta-v3-base'
    max_length = 128
    hidden_dropout_prob = 0.
    attention_probs_dropout_prob = 0.
    num_labels = 2
    batch_size = 32
    
    model_folder = '/dbfs/model_outputs'
    config = AutoConfig.from_pretrained(model_type, hidden_dropout_prob=hidden_dropout_prob, attention_probs_dropout_prob=attention_probs_dropout_prob, num_labels=num_labels)
    model = AutoModelForSequenceClassification.from_pretrained(model_folder, config=config).to(device)
    ######
    
    # initiate the lists to be used to store the predicted values
    ######
    ids = []
    labels = []
    predicted_labels = []
    probs = []
    ######
    
    # we create the PyTorch DataLoader object from the Petastorm SparkDatasetConverter object we defined earlier
    with converter_test.make_torch_dataloader(cur_shard=rank, shard_count=world_size, batch_size=batch_size, num_epochs=1) as test_dataloader:
        
        # here is the loop where we iterate over the testing data and use the fine-tuned model to get predictions
        ######
        start_time = time.time()

        for _, batch in enumerate(test_dataloader):
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            id = batch['id']
            label = batch['label'].to(device)
            
            outputs = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, labels=label)
            prob = torch.softmax(outputs.logits, dim=1).tolist()
            predicted_label = list(map(lambda x: 0 if x[1] < 0.5 else 1, prob))
            
            ids += id.tolist()
            labels += label.tolist()
            probs += prob
            predicted_labels += predicted_label
        
        end_time = time.time()
        log_to_driver('\nWorker %i: Duration: %f' % (rank, round(end_time-start_time, 3)))
        ######
        
        # we create a pandas dataframe with the ids and labels from the testing data, and the corresponding predicted labels and probabilities
        # then we aggregate the dataframes of all worker nodes in a list and save it as a pickle file
        # we also save the average model scoring duration across all worker nodes
        ######
        data_pred = {'id': ids, 'label': labels, 'predicted_label': predicted_labels, 'prob': probs}
        df_pred = pd.DataFrame(data_pred)
        df_pred = hvd.allgather_object(df_pred, name='df_pred')
        duration = hvd.allreduce(torch.tensor(round(end_time-start_time, 3))).detach().numpy()
        
        if rank == 0:
            with open(inference_folder + '/df_pred.pkl', 'wb') as f:
                pickle.dump(df_pred, f)
            with open(inference_folder + '/duration.pkl', 'wb') as f:
                pickle.dump(duration, f)
        ######

We use the [*HorovodRunner*](https://docs.microsoft.com/en-us/azure/databricks/applications/machine-learning/train-model/distributed-training/horovod-runner) class to submit the Spark job that runs the model inference function with *Horovod*.

In [0]:
hr = HorovodRunner(np=num_workers)
hr.run(model_inference)

HorovodRunner will only stream logs generated by :func:`sparkdl.horovod.log_to_driver` or
:class:`sparkdl.horovod.tensorflow.keras.LogCallback` to notebook cell output. If want to stream all
logs to driver for debugging, you can set driver_log_verbosity to 'all', like `HorovodRunner(np=2,
driver_log_verbosity='all')`.
The global names read or written to by the pickled function are {'hvd': None, 'os': None, 'log_to_driver': None, 'torch': None, 'random': None, 'np': None, 'AutoConfig': None, 'AutoModelForSequenceClassification': None, 'converter_test': None, 'time': None, 'enumerate': None, 'list': None, 'map': None, 'round': None, 'pd': None, 'open': None, 'pickle': None}.
The pickled object size is 6088 bytes.

### How to enable Horovod Timeline? ###
HorovodRunner has the ability to record the timeline of its activity with Horovod  Timeline. To
record a Horovod Timeline, set the `HOROVOD_TIMELINE` environment variable  to the location of the
timeline file to be created. You can then o

Load the list of *DataFrame* objects with model predictions and concatenete them.

In [0]:
inference_folder = '/dbfs/inference_outputs'
with open(inference_folder + '/df_pred.pkl', 'rb') as f:
    df_pred_list = pickle.load(f)

df_pred = pd.DataFrame()
for df in df_pred_list:
    df_pred = df_pred.append(df)
    
df_pred.head(10)

,id,label,predicted_label,prob
0,34359738368,0,0,"[0.9365716576576233, 0.0634283795952797]"
1,34359738369,1,0,"[0.9690632820129395, 0.030936753377318382]"
2,34359738370,1,1,"[0.004493247251957655, 0.995506763458252]"
3,34359738371,0,0,"[0.9914774298667908, 0.008522618561983109]"
4,34359738372,0,0,"[0.9696173667907715, 0.03038262203335762]"
5,34359738373,0,0,"[0.8736780881881714, 0.12632186710834503]"
6,34359738374,1,1,"[0.44084158539772034, 0.5591583847999573]"
7,34359738375,1,1,"[0.004615400452166796, 0.9953846335411072]"
8,34359738376,0,0,"[0.9921553730964661, 0.007844635285437107]"
9,34359738377,1,1,"[0.015736054629087448, 0.9842639565467834]"


Delete the *Petastorm* cache files defined earlier.

In [0]:
converter_test.delete()

/databricks/python/lib/python3.9/site-packages/petastorm/fs_utils.py:88: FutureWarning: pyarrow.localfs is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  self._filesystem = pyarrow.localfs
